# Data Blending

In [406]:
import pandas as pd
import numpy as np
import pickle as pkl
from fuzzywuzzy import fuzz
from ast import literal_eval
from datetime import datetime

In [407]:
def get_fuzz_similarity(s1: str, s2: str) -> float:
    return fuzz.token_sort_ratio(s1, s2)


def union_no_alias(l1: list[str], l2: list[str]) -> list[str]:
    union = l1.copy()
    l2 = l2.copy()
    if len(l1) > 0:
        for n1 in l1:
            for n2 in l2:
                if get_fuzz_similarity(n1, n2) > 80.0:
                    l2.remove(n2)
                    break
    union.extend(l2)
    return union

Datasets

In [408]:
links = pd.read_csv("./database/ml-32m/links.csv").rename({"movieId": "MovieID", "imdbId": "ImdbID", "tmdbId": "TmdbID"}, axis=1)
movies = pd.read_csv("./database/ml-32m/movies.csv").rename({"movieId": "MovieID", "title": "Title", "genres": "Genres"}, axis=1)
imdb = pd.read_csv("./database/imdb/movie_entries.csv")
tmdb = pd.read_csv("./database/tmdb/movie_entries.csv")
ratings = pd.read_csv("./database/ml-32m/ratings.csv")

In [409]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MovieID  87585 non-null  int64  
 1   ImdbID   87585 non-null  int64  
 2   TmdbID   87533 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.0 MB


#### I. AverageRating

Target: [global_ratings.csv](../database/merged/full/global_ratings.csv).

|Columns|Source|By|
|---|---|---|
|MovieID              |MovieLens                    ||
|AverageRating        |MovieLens + IMDB + TMDB      |Global Average|
|VoteCount            |MovieLens + IMDB + TMDB      |Global Count   |

##### MovieLens

In [442]:
ml_count = ratings[["movieId", "rating"]].groupby(["movieId"]).count()
ml_avg = ratings[["movieId", "rating"]].groupby(["movieId"]).mean()
movielens_ratings = pd.DataFrame({"MovieID": ml_count.index})
movielens_ratings["MLRating"] = ml_avg.values
movielens_ratings["MLVoteCount"] = ml_count.values
movielens_ratings.head()

,MovieID,MLRating,MLVoteCount
0,1,3.897438,68997
1,2,3.275758,28904
2,3,3.139447,13134
3,4,2.845331,2806
4,5,3.059602,13154


##### IMDB

In [443]:
imdb_ratings = imdb[["ImdbID", "Rating", "VoteCount"]]
imdb_ratings = imdb_ratings.rename({
    "Rating": "ImdbRating", 
    "VoteCount": "ImdbVoteCount"
}, axis=1)
imdb_ratings.head()

,ImdbID,ImdbRating,ImdbVoteCount
0,114709,8.3,1.1M
1,114924,6.8,117K
2,113277,8.3,736K
3,112435,5.7,6.6K
4,111430,6.8,453


##### TMDB

In [444]:
tmdb_ratings = tmdb[["TmdbID", "Rating", "VoteCount"]]
tmdb_ratings = tmdb_ratings.rename({
    "Rating": "TmdbRating", 
    "VoteCount": "TmdbVoteCount"
}, axis=1)
tmdb_ratings.head()

,TmdbID,TmdbRating,TmdbVoteCount
0,862,8.0,18326.0
1,47475,4.0,55.0
2,9614,6.6,2622.0
3,2292,7.4,2465.0
4,19819,6.1,175.0


##### Merged

In [445]:
agg_ratings = pd.merge(links, movielens_ratings, on="MovieID", how="left")
agg_ratings = pd.merge(agg_ratings, imdb_ratings, on="ImdbID", how="left").drop(["ImdbID"], axis=1)
agg_ratings = pd.merge(agg_ratings, tmdb_ratings, on="TmdbID", how="left").drop(["TmdbID"], axis=1)
agg_ratings.head()

,MovieID,MLRating,MLVoteCount,ImdbRating,ImdbVoteCount,TmdbRating,TmdbVoteCount
0,1,3.897438,68997.0,8.3,1.1M,8.0,18326.0
1,2,3.275758,28904.0,7.1,387K,7.2,10466.0
2,3,3.139447,13134.0,6.7,30K,6.5,375.0
3,4,2.845331,2806.0,6.0,13K,6.3,162.0
4,5,3.059602,13154.0,6.1,42K,6.2,725.0


In [446]:
agg_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MovieID        87585 non-null  int64  
 1   MLRating       84432 non-null  float64
 2   MLVoteCount    84432 non-null  float64
 3   ImdbRating     87433 non-null  float64
 4   ImdbVoteCount  87433 non-null  object 
 5   TmdbRating     86495 non-null  float64
 6   TmdbVoteCount  86495 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.7+ MB


In [447]:
def convert_ml_rating(row):
    ml_rate, ml_vote = row[["MLRating", "MLVoteCount"]]
    ml_rate = 0.5 if np.isnan(ml_rate) else float(ml_rate)
    ml_vote = 0 if np.isnan(ml_vote) else int(ml_vote)
    return ml_rate, ml_vote


def convert_imdb_ratings(row):
    mapping = {"K": 1e3, "M": 1e6, "B": 1e9}
    imdb_rate, imdb_vote = row[["ImdbRating", "ImdbVoteCount"]]
    imdb_rate = 1.0 if np.isnan(imdb_rate) else float(imdb_rate)
    if isinstance(imdb_vote, str):
        if imdb_vote[-1].isalpha():
            imdb_vote = float(imdb_vote[:-1]) * mapping[imdb_vote[-1]]
        imdb_vote = int(imdb_vote)
    else:
        imdb_vote = 0
    return imdb_rate, imdb_vote


def convert_tmdb_rating(row):
    tmdb_rate, tmdb_vote = row[["TmdbRating", "TmdbVoteCount"]]
    tmdb_rate = 0 if np.isnan(tmdb_rate) else float(tmdb_rate)
    tmdb_vote = 0 if np.isnan(tmdb_vote) else int(tmdb_vote)
    return tmdb_rate, tmdb_vote


def merge_rating(row):
    movie_id = row["MovieID"]
    ml_rate, ml_vote = convert_ml_rating(row)           # 0.5 - 5.0
    imdb_rate, imdb_vote = convert_imdb_ratings(row)    # 1.0 - 10.0
    tmdb_rate, tmdb_vote = convert_tmdb_rating(row)     # 0.0 - 10.0
    weights = [ml_vote, imdb_vote, tmdb_vote]
    vote_count = sum(weights)
    if vote_count:
        avg_rating = np.average(a=[ml_rate, imdb_rate / 2, 0.5 + tmdb_rate * 4.5 / 10], weights=weights)
    else:
        avg_rating = 0.0
    return pd.Series(
        [movie_id, avg_rating, vote_count], 
        index=["MovieID", "AverageRating", "VoteCount"]
    )

merged_ratings = agg_ratings.apply(merge_rating, axis=1).astype({"MovieID": "int64"})
merged_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MovieID        87585 non-null  int64  
 1   AverageRating  87585 non-null  float64
 2   VoteCount      87585 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.0 MB


In [ ]:
merged_ratings.to_csv("./database/merged/full/global_ratings.csv", index=False)

#### II. Metadata

Target: [metadatas.csv](../database/merged/full/metadatas.csv).

|Columns|Source|Priority|
|---|---|---|
|MovieID, Name              |MovieLens      ||
|Runtime                    |IMDB + TMDB    |IMDB|
|ReleaseDate                |IMDB + TMDB    |Earliest, Most Complete||
|Genres                     |MovieLens      ||
|Directors, Cast            |IMDB + TMDB    |IMDB|
|OriginCountry, Languages   |IMDB + TMDB    |IMDB|
|Plot                       |IMDB + TMDB    |IMDB|

##### ReleaseDate

In [416]:
agg_releasedate = pd.merge(links,imdb[["ImdbID", "ReleaseDate"]], on="ImdbID", how="left")
agg_releasedate = agg_releasedate.drop(["ImdbID"], axis=1).rename({"ReleaseDate": "IMDB"}, axis=1)
agg_releasedate = pd.merge(agg_releasedate, tmdb[["TmdbID", "ReleaseDate"]], on="TmdbID", how="left")
agg_releasedate = agg_releasedate.drop(["TmdbID"], axis=1).rename({"ReleaseDate": "TMDB"}, axis=1)
agg_releasedate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     87567 non-null  object
 2   TMDB     86443 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [417]:
def convert_to_date(date: str) -> tuple[datetime | None, int]:
    if isinstance(date, str):
        args_count = len(date.split("-"))
        if args_count == 1:
            return datetime.strptime(date, "%Y"), 1
        elif args_count == 2:
            return datetime.strptime(date, "%Y-%m"), 2
        else:
            return datetime.strptime(date, "%Y-%m-%d"), 3
    else:
        return datetime(9999, 12, 31), 0

In [418]:
def merge_releasedate(row):
    movie_id, imdb, tmdb = row
    imdb, p1 = convert_to_date(imdb)
    tmdb, p2 = convert_to_date(tmdb)
    if p1 == 0 and p2 == 0:
        release_date = None
    else:
        if p1 > p2:
            release_date = imdb
        elif p1 < p2:
            release_date = tmdb
        else:
            release_date = min(imdb, tmdb)
        release_date = release_date.strftime("%Y-%m-%d")
    return pd.Series(
        [int(movie_id), release_date],
        index=["MovieID", "ReleaseDate"],
    )
merged_releasedate = (agg_releasedate
    .apply(merge_releasedate, axis=1)
    .astype({"MovieID": "int64", "ReleaseDate": "object"})
    )
merged_releasedate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MovieID      87585 non-null  int64 
 1   ReleaseDate  87577 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Runtime

In [421]:
agg_runtime = pd.merge(links, imdb[["ImdbID", "Runtime"]], on="ImdbID", how="left")
agg_runtime = agg_runtime.drop(["ImdbID"], axis=1).rename({"Runtime": "IMDB"}, axis=1)
agg_runtime = pd.merge(agg_runtime, tmdb[["TmdbID", "Runtime"]], on="TmdbID", how="left")
agg_runtime = agg_runtime.drop(["TmdbID"], axis=1).rename({"Runtime": "TMDB"}, axis=1)
agg_runtime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     87061 non-null  object
 2   TMDB     86495 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [422]:
def merge_runtime(row):
    movie_id, imdb, tmdb = row
    release_date = imdb if isinstance(imdb, str) else tmdb
    return pd.Series(
        [int(movie_id), release_date],
        index=["MovieID", "Runtime"],
    )
merged_runtime = (agg_runtime
    .apply(merge_runtime, axis=1)
    .astype({"MovieID": "int64", "Runtime": "object"})
    )
merged_runtime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   Runtime  87530 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Genres

In [423]:
movie_genres = movies[["MovieID", "Genres"]]
movie_genres = movie_genres.apply(
    lambda row: pd.Series(
        [row["MovieID"], str(str(row["Genres"]).split("|"))], index=["MovieID", "Genres"]
    ),
    axis=1,
)
movie_genres.head()

,MovieID,Genres
0,1,"['Adventure', 'Animation', 'Children', 'Comedy..."
1,2,"['Adventure', 'Children', 'Fantasy']"
2,3,"['Comedy', 'Romance']"
3,4,"['Comedy', 'Drama', 'Romance']"
4,5,['Comedy']


In [424]:
movie_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   Genres   87585 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Plot

In [425]:
agg_plot = pd.merge(links, imdb[["ImdbID", "Plot"]], on="ImdbID", how="left")
agg_plot = agg_plot.drop(["ImdbID"], axis=1).rename({"Plot": "IMDB"}, axis=1)
agg_plot = pd.merge(agg_plot, tmdb[["TmdbID", "Plot"]], on="TmdbID", how="left")
agg_plot = agg_plot.drop(["TmdbID"], axis=1).rename({"Plot": "TMDB"}, axis=1)
agg_plot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     86412 non-null  object
 2   TMDB     85844 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [426]:
def merge_plot(row):
    movie_id, imdb, tmdb = row
    plot = imdb if isinstance(imdb, str) else tmdb
    if isinstance(plot, str):
        plot = plot.replace("\n", " ")
    return pd.Series(
        [int(movie_id), plot],
        index=["MovieID", "Plot"],
    )
merged_plot = agg_plot.apply(merge_plot, axis=1).astype(
    {"MovieID": "int64", "Plot": "object"}
)
merged_plot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   Plot     87389 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Directors

In [427]:
agg_directors = pd.merge(links, imdb[["ImdbID", "Directors"]], on="ImdbID", how="left")
agg_directors = agg_directors.drop(["ImdbID"], axis=1).rename({"Directors": "IMDB"}, axis=1)
agg_directors = pd.merge(agg_directors, tmdb[["TmdbID", "Directors"]], on="TmdbID", how="left")
agg_directors = agg_directors.drop(["TmdbID"], axis=1).rename({"Directors": "TMDB"}, axis=1)
agg_directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     87585 non-null  object
 2   TMDB     87533 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [428]:
def merge_directors(row):
    movie_id, imdb, tmdb = row
    if imdb != tmdb:
        imdb = literal_eval(imdb) if isinstance(imdb, str) else []
        tmdb = literal_eval(tmdb) if isinstance(tmdb, str) else []
        directors = union_no_alias(imdb, tmdb)
    else:
        directors = literal_eval(imdb) if isinstance(imdb, str) else []
    return pd.Series(
        [int(movie_id), str(directors)],
        index=["MovieID", "Directors"],
    )

merged_directors = agg_directors.apply(merge_directors, axis=1).astype(
    {"MovieID": "int64", "Directors": "object"}
)
merged_directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MovieID    87585 non-null  int64 
 1   Directors  87585 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Cast

In [429]:
agg_cast = pd.merge(links, imdb[["ImdbID", "Cast"]], on="ImdbID", how="left")
agg_cast = agg_cast.drop(["ImdbID"], axis=1).rename({"Cast": "IMDB"}, axis=1)
agg_cast = pd.merge(agg_cast, tmdb[["TmdbID", "Cast"]], on="TmdbID", how="left")
agg_cast = agg_cast.drop(["TmdbID"], axis=1).rename({"Cast": "TMDB"}, axis=1)
agg_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     85320 non-null  object
 2   TMDB     87533 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [430]:
def merge_cast(row):
    movie_id, imdb, tmdb = row
    if imdb != tmdb:
        imdb = literal_eval(imdb) if isinstance(imdb, str) else []
        tmdb = literal_eval(tmdb) if isinstance(tmdb, str) else []
        cast = union_no_alias(imdb, tmdb)
    else:
        cast = literal_eval(imdb) if isinstance(imdb, str) else []
    return pd.Series(
        [int(movie_id), str(cast)],
        index=["MovieID", "Cast"],
    )


merged_cast = agg_cast.apply(merge_cast, axis=1).astype(
    {"MovieID": "int64", "Cast": "object"}
)
merged_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   Cast     87585 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Countries

In [431]:
agg_countries = pd.merge(links, imdb[["ImdbID", "OriginCountries"]], on="ImdbID", how="left")
agg_countries = agg_countries.drop(["ImdbID"], axis=1).rename({"OriginCountries": "IMDB"}, axis=1)
agg_countries = pd.merge(agg_countries, tmdb[["TmdbID", "OriginCountries"]], on="TmdbID", how="left")
agg_countries = agg_countries.drop(["TmdbID"], axis=1).rename({"OriginCountries": "TMDB"}, axis=1)
agg_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     87582 non-null  object
 2   TMDB     87533 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [432]:
missing_imdb_countries = agg_countries[(agg_countries["IMDB"] == "[]") ^ agg_countries["IMDB"].isna()]
abbr_tmdb = set()
for _, row in missing_imdb_countries.iterrows():
    tmdb_c = row["TMDB"]
    tmdb_c = literal_eval(tmdb_c) if isinstance(tmdb_c, str) else []
    abbr_tmdb = abbr_tmdb.union(set(tmdb_c))
abbr_tmdb

{'AU',
 'BR',
 'CA',
 'CN',
 'CZ',
 'DE',
 'DK',
 'ES',
 'FR',
 'GB',
 'IE',
 'IN',
 'IT',
 'JP',
 'KE',
 'KR',
 'NZ',
 'RU',
 'SE',
 'US'}

In [433]:
countries_mapper = {
    "AU": "Australia",
    "BR": "Brazil",
    "CA": "Canada",
    "CN": "China",
    "CZ": "Czech Republic",
    "DE": "Germany",
    "DK": "Denmark",
    "ES": "Spain",
    "FR": "France",
    "GB": "United Kingdom",
    "IE": "Ireland",
    "IN": "India",
    "IT": "Italy",
    "JP": "Japan",
    "KE": "Kenya",
    "KR": "South Korea",
    "NZ": "New Zealand",
    "RU": "Russia",
    "SE": "Sweden",
    "US": "United States",
}

In [434]:
def merge_countries(row):
    m_id, imdb, tmdb = row
    imdb = literal_eval(imdb) if isinstance(imdb, str) else []
    tmdb = literal_eval(tmdb) if isinstance(tmdb, str) else []
    unique = set(imdb)
    for lang in tmdb:
        if lang in countries_mapper:
            lang = countries_mapper[lang]
            if lang in unique:
                continue
            imdb.append(lang)
    return pd.Series([m_id, str(imdb)], ["MovieID", "OriginCountries"])

merged_countries = agg_countries.apply(merge_countries, axis=1).astype({"MovieID": "int64"})
merged_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MovieID          87585 non-null  int64 
 1   OriginCountries  87585 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Languages

In [435]:
agg_langs = pd.merge(links, imdb[["ImdbID", "Languages"]], on="ImdbID", how="left")
agg_langs = agg_langs.drop(["ImdbID"], axis=1).rename({"Languages": "IMDB"}, axis=1)
agg_langs = pd.merge(agg_langs, tmdb[["TmdbID", "Languages"]], on="TmdbID", how="left")
agg_langs = agg_langs.drop(["TmdbID"], axis=1).rename({"Languages": "TMDB"}, axis=1)
agg_langs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  87585 non-null  int64 
 1   IMDB     87582 non-null  object
 2   TMDB     87533 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


Remove/Convert specific languages

In [436]:
imdb_languages_remove = [
    "Sign Language",
    "Aboriginal",
    "languages",
    "Ancient",
    "-",
    "Kru",
    "Low ",
    "Middle ",
    "Western ",
    " Old",
    "Old ",
]
tmdb_languages_remove = ["Raeto-Romance"]
imdb_convert_languages_map = {
    "Chinese": "Mandarin",
    "Central Khmer": "Khmer",
    "Irish Gaelic": "Irish",
    "Aromanian": "Portuguese",
    "Crimean Tatar": "Tatar",
    "Ibo": "Igbo",
    "Tonga (Tonga Islands)": "Tonga",
    "None": "No Language",
}
tmdb_convert_languages_map = {
    "Kalaallisut": "Greenlandic",
    "Haitian; Haitian Creole": "Haitian",
    "Chichewa; Nyanja": "Nyanja",
    "Eastern Frisian": "Frisian",
    "Serbo-Croatian": "Croatian",
    "Norwegian Bokmål": "Norwegian",
    "Ossetian; Ossetic": "Ossetian",
}

In [437]:
def remove_specific_languages(row):
    m_id, imdb, tmdb = row
    new_imdb = []
    if isinstance(imdb, str):
        imdb = literal_eval(imdb)
        for lang in imdb:
            for s_lang in imdb_languages_remove:
                if lang.find(s_lang) >= 0:
                    break
            else:
                if lang in imdb_convert_languages_map:
                    lang = imdb_convert_languages_map[lang]
                new_imdb.append(lang)
    new_tmdb = []
    if isinstance(tmdb, str):
        tmdb = literal_eval(tmdb)
        for lang in tmdb:
            for s_lang in tmdb_languages_remove:
                if lang.find(s_lang) >= 0:
                    break
            else:
                if lang in tmdb_convert_languages_map:
                    lang = tmdb_convert_languages_map[lang]
                new_tmdb.append(lang)
    imdb = str(new_imdb)
    tmdb = str(new_tmdb)
    return pd.Series([m_id, imdb, tmdb], agg_langs.columns)

agg_langs = agg_langs.apply(remove_specific_languages, axis=1)

In [438]:
def merge_languages(row):
    m_id, imdb, tmdb = row
    imdb: list = literal_eval(imdb)
    tmdb: list = literal_eval(tmdb)
    unique = set(imdb)
    for lang in tmdb:
        # Ensure languages priority
        if lang in unique:
            continue
        imdb.append(lang)
    return pd.Series([m_id, str(imdb)], ["MovieID", "Languages"])

merged_langs = agg_langs.apply(merge_languages, axis=1).astype({"MovieID": "int64"})
merged_langs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MovieID    87585 non-null  int64 
 1   Languages  87585 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


##### Final merged

In [439]:
merged_metadata = movies[["MovieID", "Title"]]
merged_metadata = pd.merge(merged_metadata, merged_runtime, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, merged_releasedate, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, movie_genres, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, merged_directors, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, merged_cast, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, merged_countries, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, merged_langs, on="MovieID", how="left")
merged_metadata = pd.merge(merged_metadata, merged_plot, on="MovieID", how="left")
merged_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MovieID          87585 non-null  int64 
 1   Title            87585 non-null  object
 2   Runtime          87530 non-null  object
 3   ReleaseDate      87577 non-null  object
 4   Genres           87585 non-null  object
 5   Directors        87585 non-null  object
 6   Cast             87585 non-null  object
 7   OriginCountries  87585 non-null  object
 8   Languages        87585 non-null  object
 9   Plot             87389 non-null  object
dtypes: int64(1), object(9)
memory usage: 6.7+ MB


In [ ]:
merged_metadata.to_csv("./database/merged/full/metadatas.csv", index=False)